# Greedy Blend Selection

Starting from the best single model (by holdout MAE), incrementally add models
and report whether each addition improves the inverse-MAE-weighted blend.
Uses the production `blend_config.json` and the same methodology as `train_and_blend()`.

In [ ]:
from startup import PROJ_ROOT

import json

import joblib
import mlflow
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score

from src.config import MLFLOW_TRACKING_URI
from src.features.preprocessors import load_dataset
from src.modeling.blend import (
    BLEND_CONFIG_PATH,
    PRODUCTION_DIR,
    _compute_inverse_mae_weights,
    _flatten_daily_to_hourly,
    _flatten_y_true,
)
from src.config.modeling import BLEND_HOLDOUT_DAYS

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

with open(BLEND_CONFIG_PATH) as f:
    config = json.load(f)

holdout_days = config["holdout_days"]
models_info = config["models"]
print(f"Loaded {len(models_info)} models, holdout_days={holdout_days}")
for m in models_info:
    print(f"  {m['name']:20s}  MAE={m['holdout_mae']:.2f}  w={m['weight']:.4f}")

## Load models and generate holdout predictions

In [ ]:
# Load datasets (cached by dataset_run_id)
dataset_cache: dict[str, tuple] = {}
for info in models_info:
    rid = info["dataset_run_id"]
    if rid not in dataset_cache:
        print(f"Loading dataset {rid[:8]}...")
        X, y_df, _ = load_dataset(run_id=rid)
        dataset_cache[rid] = (X, y_df.values)

# Generate holdout predictions per model
model_names = []
holdout_preds = {}  # name -> hourly preds array
holdout_maes = {}   # name -> scalar MAE
ref_y = None        # common hourly y_true (from first model's holdout)

for info in models_info:
    name = info["name"]
    group_size = info["group_size"]
    X, y = dataset_cache[info["dataset_run_id"]]

    holdout_rows = holdout_days * group_size
    split_idx = len(X) - holdout_rows
    X_holdout = X.iloc[split_idx:]
    y_holdout = y[split_idx:]

    pipeline = joblib.load(PRODUCTION_DIR / info["file"])
    y_pred = pipeline.predict(X_holdout)

    y_pred_flat = _flatten_daily_to_hourly(y_pred)
    y_true_flat = _flatten_y_true(y_holdout, group_size)

    # Align lengths
    min_len = min(len(y_pred_flat), len(y_true_flat))
    y_pred_flat = y_pred_flat[:min_len]
    y_true_flat = y_true_flat[:min_len]

    holdout_preds[name] = y_pred_flat
    holdout_maes[name] = mean_absolute_error(y_true_flat, y_pred_flat)
    model_names.append(name)

    if ref_y is None:
        ref_y = y_true_flat

    print(f"  {name:20s}  holdout MAE={holdout_maes[name]:.4f}  shape={y_pred_flat.shape}")

# Align all predictions to common length (inner join on hourly index)
common_len = min(len(ref_y), *(len(p) for p in holdout_preds.values()))
ref_y = ref_y[:common_len]
for name in model_names:
    holdout_preds[name] = holdout_preds[name][:common_len]

print(f"\nCommon holdout length: {common_len} hours ({common_len // 24} days)")

## Greedy forward selection

In [ ]:
def blend_metrics(names: list[str]) -> dict:
    """Compute blend metrics for a subset of models using inverse-MAE weights."""
    maes = np.array([holdout_maes[n] for n in names])
    weights = _compute_inverse_mae_weights(maes)
    preds_matrix = np.column_stack([holdout_preds[n] for n in names])
    y_blend = preds_matrix @ weights
    return {
        "mae": mean_absolute_error(ref_y, y_blend),
        "rmse": float(np.sqrt(np.mean((ref_y - y_blend) ** 2))),
        "r2": float(r2_score(ref_y, y_blend)),
        "weights": dict(zip(names, weights.round(4))),
    }


# Rank models by individual holdout MAE (best first)
ranked = sorted(model_names, key=lambda n: holdout_maes[n])

print("Individual model ranking (by holdout MAE):")
print(f"{'Rank':<6}{'Model':<22}{'MAE':>10}")
print("-" * 38)
for i, name in enumerate(ranked, 1):
    print(f"{i:<6}{name:<22}{holdout_maes[name]:>10.4f}")

# Greedy forward selection
print("\n" + "=" * 80)
print("Greedy Forward Selection")
print("=" * 80)
print(
    f"{'Step':<6}{'Added Model':<22}{'Blend MAE':>12}"
    f"{'Delta':>10}{'Blend RMSE':>12}{'R2':>8}{'N':>4}"
)
print("-" * 74)

selected = []
remaining = list(ranked)
prev_mae = None
results = []

for step in range(1, len(ranked) + 1):
    # Try adding each remaining model and pick the one that gives best blend MAE
    best_candidate = None
    best_metrics = None

    for candidate in remaining:
        trial = selected + [candidate]
        m = blend_metrics(trial)
        if best_metrics is None or m["mae"] < best_metrics["mae"]:
            best_candidate = candidate
            best_metrics = m

    selected.append(best_candidate)
    remaining.remove(best_candidate)

    delta = best_metrics["mae"] - prev_mae if prev_mae is not None else 0.0
    delta_str = f"{delta:+.4f}" if prev_mae is not None else "--"
    marker = " *" if prev_mae is not None and delta < 0 else ""

    print(
        f"{step:<6}{best_candidate:<22}{best_metrics['mae']:>12.4f}"
        f"{delta_str:>10}{best_metrics['rmse']:>12.4f}"
        f"{best_metrics['r2']:>8.4f}{len(selected):>4}{marker}"
    )

    results.append({
        "step": step,
        "added": best_candidate,
        "blend_mae": best_metrics["mae"],
        "delta_mae": delta if prev_mae is not None else None,
        "blend_rmse": best_metrics["rmse"],
        "blend_r2": best_metrics["r2"],
        "n_models": len(selected),
        "models": list(selected),
        "weights": best_metrics["weights"],
    })
    prev_mae = best_metrics["mae"]

print("\n* = improvement over previous step")

# Find optimal subset
best_step = min(results, key=lambda r: r["blend_mae"])
print(f"\nBest blend: step {best_step['step']} with {best_step['n_models']} models")
print(f"  MAE  = {best_step['blend_mae']:.4f}")
print(f"  RMSE = {best_step['blend_rmse']:.4f}")
print(f"  R2   = {best_step['blend_r2']:.4f}")
print(f"  Models: {best_step['models']}")
print(f"  Weights: {best_step['weights']}")

In [ ]:
# Summary table
summary = pd.DataFrame(results)
summary = summary[["step", "added", "n_models", "blend_mae", "delta_mae", "blend_rmse", "blend_r2"]]
summary["delta_mae"] = summary["delta_mae"].map(lambda x: f"{x:+.4f}" if x is not None else "--")
summary